<a href="https://colab.research.google.com/github/lucasthaynan/servidores-camara-maceio/blob/main/web_scraping_servidores_camara_maceio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Raspando dados das páginas de remuneração dos servidores da Câmara Municipal de Maceió (AL)
https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios

Instalando a biblioteca `tdqm` que permite adicionar uma barra de progresso ao executar um for, além da `google.auth` para acesso as APIs do Google e `gspread` que conecta com o Google Sheets

In [ ]:
!pip install tqdm

In [ ]:
!pip install --upgrade gspread

In [ ]:
!pip install google.auth

In [3]:
import requests
import re
import pandas as pd

from tqdm import tqdm
import gspread
from google.oauth2.service_account import Credentials

Acessando a lista de páginas e coletando os links referentes a cada pagamento mensal dos servidores. Com isso, gera uma lista com todos os links

In [4]:

def pega_links_servidores(paginas=1):
  """Função que percorre as páginas e gera uma lista de links dos servidores""" # docstring
  
  lista_paginacao = list(range(1, (paginas + 1))) # cria sequência de números [1, 2, 3,...]

  lista_links = []
  for pagina in tqdm(lista_paginacao):
    url = f"https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios&pagina={pagina}"
    resposta = requests.get(url)
    html = resposta.text

    links_pagina = re.findall("location.href = '([^']+)';", html)
    lista_links = links_pagina + lista_links
    
  return lista_links
  

In [5]:
# chamando a função e passando a como parâmetro a quantidade de páginas da requisição

todos_os_links = pega_links_servidores(2)

todos_os_links

100%|██████████| 2/2 [00:03<00:00,  1.99s/it]


['https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=19&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=100674&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=92187&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=100938&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=92171&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=101024&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=100770&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=92178&mes=9&ano=2021',
 'https://www.maceio.al.leg.br/transparencia/portal/salarios-subsidios-detalhes&matricula=10089

Acessando cada uma das páginas dos servidores e coletando as informações de pagamento

In [6]:

def raspador():
  """Função que percorre os links da lista e extrai as informações dos servidores"""

  def convert_moeda_float(valor):
    """Função que transforma os valores em float"""

    valor = valor[1].replace("R$ ", "").replace(".", "").replace(",", ".")
    valor = float(valor)
    return valor

  
  lista_pag_servidores = []

  # percorrendo a lista de links e extraindo o html  
  for link in tqdm(todos_os_links): 
    resposta = requests.get(link)
    html = resposta.text
    partes = html.split("</td>") # divindo o html em partes e salvando em uma lista "partes"

    # salvando valores em variáveis
    matricula = partes[1].split("td>")     
    referencia = partes[3].split("td>") 
    vinculo = partes[5].split("td>")
    servidor = partes[7].split("td>")
    cargo = partes[9].split("td>")
    cpf = partes[11].split("td>")
    lotacao = partes[13].split("td>")

    remuneracao = partes[15].split("td>")
    remuneracao = convert_moeda_float(partes[15].split("td>"))

    abono = partes[17].split("td>")
    abono = convert_moeda_float(partes[17].split("td>"))

    eventuais = partes[19].split("td>")
    eventuais = convert_moeda_float(partes[19].split("td>"))

    desconto = partes[21].split("td>")
    desconto = convert_moeda_float(desconto)

    salario_liquido = partes[23].split("td>")
    salario_liquido = convert_moeda_float(salario_liquido)

    lista_pag_servidores.append({
      "matricula": matricula[1],
      "referencia": referencia[1],
      "vinculo": vinculo[1],
      "servidor": servidor[1],
      "cargo": cargo[1],
      "cpf": cpf[1],
      "lotacao": lotacao[1],
      "remuneracao": remuneracao,
      "abono": abono,
      "eventuais": eventuais,
      "desconto": desconto,
      "salario_liquido": salario_liquido
    }) 
  
  return lista_pag_servidores



In [ ]:
dados_servidores = raspador()
dados_servidores

### Converter lista de dicionários em DataFrame

In [8]:
df_servidores = pd.DataFrame(dados_servidores)

df_servidores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   matricula        40 non-null     object 
 1   referencia       40 non-null     object 
 2   vinculo          40 non-null     object 
 3   servidor         40 non-null     object 
 4   cargo            40 non-null     object 
 5   cpf              40 non-null     object 
 6   lotacao          40 non-null     object 
 7   remuneracao      40 non-null     float64
 8   abono            40 non-null     float64
 9   eventuais        40 non-null     float64
 10  desconto         40 non-null     float64
 11  salario_liquido  40 non-null     float64
dtypes: float64(5), object(7)
memory usage: 3.9+ KB


In [9]:
df_servidores.head()

,matricula,referencia,vinculo,servidor,cargo,cpf,lotacao,remuneracao,abono,eventuais,desconto,salario_liquido
0,00000019,09 / 2021,ESTATUTARIO,ALECY AMORIM DA SILVA,TECNICO LEGISLATIVO,***66468,CAMARA MUNICIPAL DE MACEIO,6168.6,0.0,0.0,1483.95,4684.65
1,00100674,09 / 2021,CARGO COMISSIONADO,ALESSANDRA LOUREIRO SIQUEIRA DE LEMOS,SECRETARIO PARLAMENTAR,***75440,COMISSIONADO - GABINETE,8000.0,0.0,0.0,1875.83,6124.17
2,00092187,09 / 2021,CARGO COMISSIONADO,ALESSANDRA TABOZA BARROS MOREIRA,CARGOS DE NATUREZA ESPECIAL,***04482,MESA,5000.0,0.0,0.0,916.12,4083.88
3,00100938,09 / 2021,CARGO COMISSIONADO,ALESSANDRO QUINTELA BRANDAO DE GUSMAO,SECRETARIO PARLAMENTAR,***89491,COMISSIONADO - GABINETE,8000.0,0.0,0.0,1875.83,6124.17
4,00092171,09 / 2021,CARGO COMISSIONADO,ALEX DE OLIVEIRA CAVALCANTE,CARGOS DE NATUREZA ESPECIAL,***51419,MESA,5000.0,0.0,0.0,916.12,4083.88


### Exportando para .CSV

In [144]:
df_servidores.to_csv("salarios_servidores_camara_maceio.csv")

### **Salvando dados no Google Sheets**

In [11]:
# tutorial para pegar Json da API do Google Sheets: https://www.youtube.com/watch?v=ddf5Z0aQPzY&t=171

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file('Google Sheets API Key.json',scopes=scopes)

gc = gspread.authorize(credentials)

### criando nova planilha

In [ ]:
planilha = gc.create('Pagamentos Servidores - Câmara Maceió') 

### exportando dados para o sheets

In [14]:

# abrindo a planilha do Google Sheets com o link
planilha_servidores = gc.open("Pagamentos Servidores - Câmara Maceió")

quantidade_registros = int(len(df_servidores)) # variável com a quantidade de registros do dataframe

# criando nova página (aba) dentro da planilha aberta
nova_pagina_registros = planilha_servidores.add_worksheet(f"{quantidade_registros} registros - 10/10/21", rows=quantidade_registros, cols=12)

nova_pagina_registros.update([df_servidores.columns.values.tolist()] + df_servidores.values.tolist())

{'spreadsheetId': '1cesTMLm5JDOjm-YH96sMH3zrJMzmyGEXi_elE2SbxQY',
 'updatedCells': 492,
 'updatedColumns': 12,
 'updatedRange': "'40 registros - 10/10/21'!A1:L41",
 'updatedRows': 41}